# Interactive Moment Map Explorer

This notebook demonstrates the `moment-explorer` package for interactive moment map visualization using Plotly.

## Features

- 🔭 **Multi-Cube Explorer** - Dropdown to switch between cubes
- 📊 **Moment Maps** - M0, M1, M8, M9 with interactive controls
- 🔍 **Cube Viewer** - Channel-by-channel visualization with mask overlay
- ⚙️ **Sigma Clipping** - Optional noise filtering with smart warnings
- 💾 **FITS Export** - Save moment maps with one click

In [1]:
# # Install the package in development mode (run once)
# !pip install -e ..

In [2]:
import os
import sys

# Add the src directory to the path (for development)
sys.path.insert(0, os.path.abspath('../src'))

from moment_explorer import (
    create_interactive_explorer,
    create_multi_cube_explorer,
    create_cube_viewer,
    create_launcher_ui
)

## Configuration: Available Cubes

Define the paths to your FITS cubes and masks.

In [3]:
base_path = '/Users/jea/AATau/cleaning_data/'

# Available cubes
available_cubes = {
    'N2H+ (2013SG2)': {
        'cube': os.path.join(base_path, 'fits_2013SG2/AATau_N2H+_clean1_contsub.fits'),
        'mask': os.path.join(base_path, 'fits_2013SG2/AATau_N2H+_clean0_contsub.mask.fits'),
        'name': 'N$_2$H$^+$'
    },
    'HCN (2013SG1)': {
        'cube': os.path.join(base_path, 'fits_2013SG1/AATau_HCN_clean1_contsub.fits'),
        'mask': os.path.join(base_path, 'fits_2013SG1/AATau_HCN_clean0_contsub.mask.fits'),
        'name': 'HCN'
    },
    'HCO+ (2013SG1)': {
        'cube': os.path.join(base_path, 'fits_2013SG1/AATau_HCO+_clean1_contsub.fits'),
        'mask': os.path.join(base_path, 'fits_2013SG1/AATau_HCO+_clean0_contsub.mask.fits'),
        'name': 'HCO$^{+}$'
    },
    '13CO (2015)': {
        'cube': os.path.join(base_path, 'fits_2015/AATau_13CO_clean1_contsub.fits'),
        'mask': os.path.join(base_path, 'fits_2015/AATau_13CO_clean0_contsub.mask.fits'),
        'name': '$^{13}$CO'
    },
    'C18O (2015)': {
        'cube': os.path.join(base_path, 'fits_2015/AATau_C18O_clean1_contsub.fits'),
        'mask': os.path.join(base_path, 'fits_2015/AATau_C18O_clean0_contsub.mask.fits'),
        'name': 'C$^{18}$O'
    },
    'CN SPW1 (2015)': {
        'cube': os.path.join(base_path, 'fits_2015/AATau_CN_SPW1_clean1_contsub.fits'),
        'mask': os.path.join(base_path, 'fits_2015/AATau_CN_SPW1_clean0_contsub.mask.fits'),
        'name': 'CN J=7/2-5/2'
    },
    'CN SPW3 (2015)': {
        'cube': os.path.join(base_path, 'fits_2015/AATau_CN_SPW3_clean1_contsub.fits'),
        'mask': os.path.join(base_path, 'fits_2015/AATau_CN_SPW3_clean0_contsub.mask.fits'),
        'name': 'CN J=5/2-3/2'
    }
}

print("Available cubes:")
for key in available_cubes.keys():
    print(f"  - {key}")

Available cubes:
  - N2H+ (2013SG2)
  - HCN (2013SG1)
  - HCO+ (2013SG1)
  - 13CO (2015)
  - C18O (2015)
  - CN SPW1 (2015)
  - CN SPW3 (2015)


---

# Option 1: Multi-Cube Explorer (Recommended)

**NEW!** Switch between multiple cubes using a dropdown selector.

### Features:
- Dropdown menu to select which cube to view
- Automatic reloading when switching cubes
- All standard controls (moment type, channels, clipping, etc.)
- Sigma clipping now works for all moment types!
- Smart warnings when using clipping with M0

In [4]:
# Create multi-cube explorer with dropdown selector
explorer, ui, selector = create_multi_cube_explorer(
    available_cubes,
    enable_prefix_sums=True,
    default_cube='N2H+ (2013SG2)'  # Optional: which cube to load first
)

Prefix-sum optimization enabled for M0/M1
Note: Using standard Heatmap (Heatmapgl not available). Performance may be slower for large maps.


### 💡 Usage Tips

**Sigma Clipping Guide:**
- **M0 (Integrated Intensity)**: Set Clip σ = 0 (no clipping recommended)
  - Clipping excludes faint emission and underestimates total flux
  - Orange warning appears if you enable clipping for M0
- **M1 (Velocity)**: Set Clip σ = 3-5 (clipping recommended)
  - Excludes noisy pixels from velocity measurements
- **M8 (Peak)**: Set Clip σ = 0 (peak is already robust)
- **M9 (Linewidth)**: Set Clip σ = 3-5 (clipping recommended)
  - Prevents noise from broadening the line

**Auto-Apply:**
- Enable for smooth exploration (updates after ~200ms)
- Disable for precise control (use Apply button)

**Switching Cubes:**
- Use the dropdown at the top
- Channel sliders automatically adjust to new cube
- Moment map regenerates with same settings

---

# Option 2: Single Cube Explorer

If you only need one cube at a time:

In [6]:
# Select a cube
selected_cube = 'N2H+ (2013SG2)'

cube_info = available_cubes[selected_cube]
cube_path = cube_info['cube']
mask_path = cube_info['mask']

# Create the interactive explorer
explorer_single, ui_single = create_interactive_explorer(
    cube_path=cube_path,
    mask_path=mask_path,
    enable_prefix_sums=True
)

Cube loaded: (100, 500, 500)
Velocity range: -3000.00 to 26700.00 km/s
RMS: 3.878e-03
Prefix-sum optimization enabled for M0/M1
Note: Using standard Heatmap (Heatmapgl not available). Performance may be slower for large maps.


---

# Option 3: Cube Viewer (Channel Explorer)

**NEW!** Explore individual channels with mask overlay.

### Features:
- Channel slider to navigate through cube
- Red contour overlay showing mask boundaries
- vmin/vmax controls for intensity scaling
- Auto-scale option (1st to 99th percentile)
- Multiple colormap options

### Use Cases:
- Find emission channels before creating moment maps
- Verify mask coverage across channels
- Check for artifacts or bad channels
- Explore velocity structure

In [7]:
# Create cube viewer
viewer = create_cube_viewer(
    cube_path=cube_info['cube'],
    mask_path=cube_info['mask']
)

Cube loaded: (100, 500, 500)
Velocity range: -3000.00 to 26700.00 km/s
Number of channels: 100


### 💡 Cube Viewer Workflow

1. **Find emission channels**
   - Enable "Auto-scale intensity"
   - Move channel slider through cube
   - Note which channels show emission

2. **Verify mask**
   - Enable "Show mask contour"
   - Check that red contour matches emission
   - Look for channels where mask is too small/large

3. **Check data quality**
   - Go to line-free channels (edges)
   - Note vmax value (≈ noise level)
   - Look for artifacts or bad pixels

4. **Use in Moment Map Explorer**
   - Use found channel range in moment map explorer
   - Generate M0, M1, M8, M9 maps
   - Save to FITS

---

# Option 4: Launcher UI (File Browser)

**NEW!** Interactive file selection interface.

In [8]:
# Display launcher UI
# Fill in paths and click "Launch Explorer"
from IPython.display import display
display(create_launcher_ui())

---

# Advanced: Programmatic Usage

Use the package without the UI for batch processing or custom workflows.

In [ ]:
from moment_explorer import MomentMapExplorer
import matplotlib.pyplot as plt
import numpy as np

# Create explorer
explorer_adv = MomentMapExplorer()
explorer_adv.load_cube(cube_path, mask_path)

# Generate moment maps with different clipping
print("\n=== M0 Comparison: No Clipping vs Clipping ===")

# M0 without clipping (recommended)
M0_noclip, time_noclip = explorer_adv.generate(
    moment='M0',
    first=30,
    last=70,
    clip_sigma=0.0,  # No clipping
    use_mask=True
)

# M0 with clipping (not recommended)
M0_clip, time_clip = explorer_adv.generate(
    moment='M0',
    first=30,
    last=70,
    clip_sigma=3.0,  # With clipping
    use_mask=True
)

# Compare
total_flux_noclip = np.nansum(M0_noclip)
total_flux_clip = np.nansum(M0_clip)
flux_lost = (1 - total_flux_clip / total_flux_noclip) * 100

print(f"\nNo clipping: Total flux = {total_flux_noclip:.3e}")
print(f"With clipping (3σ): Total flux = {total_flux_clip:.3e}")
print(f"Flux lost to clipping: {flux_lost:.1f}%")
print(f"\n⚠️  This is why clipping is NOT recommended for M0!")

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

im0 = axes[0].imshow(M0_noclip, origin='lower', cmap='viridis')
plt.colorbar(im0, ax=axes[0], label='Integrated Intensity')
axes[0].set_title(f'M0 - No Clipping\nTotal Flux: {total_flux_noclip:.3e}')

im1 = axes[1].imshow(M0_clip, origin='lower', cmap='viridis')
plt.colorbar(im1, ax=axes[1], label='Integrated Intensity')
axes[1].set_title(f'M0 - With 3σ Clipping\nTotal Flux: {total_flux_clip:.3e}\n({flux_lost:.1f}% lost)')

plt.tight_layout()
plt.show()

## Generate All Moment Types

In [ ]:
# Generate all moment types with recommended settings
moments_to_generate = [
    {'moment': 'M0', 'clip_sigma': 0.0, 'label': 'Integrated Intensity'},
    {'moment': 'M1', 'clip_sigma': 3.0, 'label': 'Velocity [km/s]'},
    {'moment': 'M8', 'clip_sigma': 0.0, 'label': 'Peak Intensity'},
    {'moment': 'M9', 'clip_sigma': 3.0, 'label': 'Linewidth [km/s]'}
]

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for idx, config in enumerate(moments_to_generate):
    moment_map, compute_time = explorer_adv.generate(
        moment=config['moment'],
        first=30,
        last=70,
        clip_sigma=config['clip_sigma'],
        use_mask=True
    )
    
    cmap = 'RdBu_r' if config['moment'] == 'M1' else 'viridis'
    im = axes[idx].imshow(moment_map, origin='lower', cmap=cmap)
    plt.colorbar(im, ax=axes[idx], label=config['label'])
    axes[idx].set_title(
        f"{config['moment']} - {config['label']}\n"
        f"Clip σ = {config['clip_sigma']} ({compute_time*1000:.1f} ms)"
    )

plt.tight_layout()
plt.show()

## Performance Comparison: Standard vs Prefix-Sum

In [ ]:
import time

# Test parameters
test_params = {
    'moment': 'M0',
    'first': 30,
    'last': 70,
    'clip_sigma': 0.0,
    'use_mask': True
}

# Standard method
explorer_std = MomentMapExplorer()
explorer_std.load_cube(cube_path, mask_path)
explorer_std.enable_prefix_sums(False)

_, time_std = explorer_std.generate(**test_params)

# Prefix-sum method
explorer_prefix = MomentMapExplorer()
explorer_prefix.load_cube(cube_path, mask_path)
explorer_prefix.enable_prefix_sums(True)

_, time_prefix = explorer_prefix.generate(**test_params)

print(f"Standard method: {time_std*1000:.1f} ms")
print(f"Prefix-sum method: {time_prefix*1000:.1f} ms")
print(f"Speedup: {time_std/time_prefix:.2f}x")

---

# Summary

This notebook demonstrates:

1. ✅ **Multi-Cube Explorer** - Dropdown selector for multiple cubes
2. ✅ **Cube Viewer** - Channel-by-channel exploration with mask overlay
3. ✅ **Sigma Clipping** - Available for all moment types with smart warnings
4. ✅ **Launcher UI** - Interactive file selection
5. ✅ **Programmatic API** - Batch processing and custom workflows

## Key Takeaways

- **M0**: Always use Clip σ = 0 (no clipping) to preserve total flux
- **M1/M9**: Use Clip σ = 3-5 to exclude noise
- **Cube Viewer**: Use to find emission channels before making moment maps
- **Prefix Sums**: Enable for 2-5x faster M0/M1 computation

## Documentation

- [README.md](../README.md) - Main documentation
- [SIGMA_CLIPPING_GUIDE.md](../SIGMA_CLIPPING_GUIDE.md) - Clipping best practices
- [CUBE_VIEWER_GUIDE.md](../CUBE_VIEWER_GUIDE.md) - Channel viewer usage
- [MULTI_CUBE_EXAMPLE.md](../MULTI_CUBE_EXAMPLE.md) - Multi-cube explorer guide